In [1]:
from vm.mnemonic import WProtectMnemonic
from vm.analysis import PEAnalysis, WProtectEmulator, WProtectTracer
import vm.obfuscation
import semantic.flow
import semantic.translator
from vm.rule_generator import generate_rules

__Load binary file and recover its instructions__  
There are several several samples provided. Those samples test conditional jumps as they do require a context. Uncomment a sample, or provide a binary of your own.

In [2]:
binary = WProtectEmulator("if_all/if_jz.wp.exe")
#binary = WProtectEmulator("if_all/if_jb.wp.exe")
#binary = WProtectEmulator("if_all/if_jo.wp.exe")
#binary = WProtectEmulator("if_all/if_js.wp.exe")
#binary = WProtectEmulator("if_all/if_jbe.wp.exe")
#binary = WProtectEmulator("if_all/if_jl.wp.exe")
#binary = WProtectEmulator("if_all/if_jle.wp.exe")
#binary = WProtectEmulator("if_all/if_jp.wp.exe")
binary.find()
instructions = binary.recover_mnemonics(binary.offset, WProtectMnemonic)[1]
WProtectMnemonic.consolidate_names(instructions)

[INFO]	PE loaded
[INFO]	reachable code analysis done


[INFO]	WProtect vm mnemonics found at offset 0x40a901


[INFO]	Mnemonics loaded


__Reconstruct the control flow and get continuous code blocks.__  
The function _callback_ passes instructions to _WProtectControlFlow_ as they are read by _WProtectTracer_. _WProtectTracer_ always creates new states (one if no branching occured, two otherwise), these states are inserted into a processing queue. In other words, after at most 2 instructions another branch, if available, is processed.  

_WProtectControlFlow_ is then utilised to reconstruct the control flow and output code blocks (a sequence of instructions that is always executed continuously, the only jump is permitted at its end; similarly, the only jump destination is permitted at its beginning).

In [7]:
def callback(offset, instruction, args, off):
    control_flow.add_node(offset, instruction, args, off)

control_flow = semantic.flow.WProtectControlFlow(instructions)
instance = WProtectTracer(binary, instructions)

machine = [instance]
index = 0
while index < len(machine):
    try:
        tmp = machine[index].step(callback)
        machine += tmp
    except semantic.flow.PositionInstructionError as err:
        pass
    index += 1
    
blocks = control_flow.compile_blocks()

__Translate blocks and assemble__  
Use _semantic.translator.translate_blocks_ to translate all the instruction into the assembly language. A label is inserted at the beginning of every block. Translated blocks are concatenated, prepended with a memory reserved for registers, and 

In [10]:
def get_code(blocks):
    def hexify(string):
        return "".join([hex(ord(char))[2:].zfill(2) for char in string])
    return hexify(semantic.translator.translate_blocks(blocks))

get_code(blocks)

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000e9dc01000058a35800000058a32800000058a31800000058a33400000058a33c00000058a36000000058a33800000058a35c00000058a300000000a12800000050b80100000050448a1c24589cb8fefbffff5058a34800000054588b005058f7d05bf7d39c58a354000000a1480000005054588b005058f7d05bf7d39c58a34c00000058f7d05bf7d39c58a330000000a15800000050b8010400005058a34800000054588b005058f7d05bf7d39c58a314000000a1480000005054588b005058f7d05bf7d39c58a31c00000058f7d05bf7d39c58a35000000058a34800000058a314000000a14800000050a1140000005058f7d05bf7d39c58a330000000a14800000050a1140000005058f7d05bf7d39c58a34000000058f7d05bf7d39c58a30c00000058a35800000058a